In [195]:
import numpy as np
from numpy import linalg as LA
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import os
import fnmatch
import re
import wexpect
import shutil
import time
#import seaborn as sns # it will set to SNS style
#sns.reset_orig() # set back to the Matplotlib style
#sns.set() # set to sns style
# set the font
#plt.rcParams["font.family"] = 'arial'
%matplotlib ipympl

from lmfit import minimize, Model, Minimizer, Parameters, Parameter, report_fit
from scipy.signal import savgol_filter

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from scipy import interpolate


"""
1. only after 'child.expect(...)' can you use child.befor or ...
2. 'child.expect()' give the output since last 'expect' till this one
3. If problems to call SPECTRE and using wexpect, in the take manager, end the process of spectre and a cmd window. 
4. add a "expect" after spectre started fitting to wait
"""

'\n1. only after \'child.expect(...)\' can you use child.befor or ...\n2. \'child.expect()\' give the output since last \'expect\' till this one\n3. If problems to call SPECTRE and using wexpect, in the take manager, end the process of spectre and a cmd window. \n4. add a "expect" after spectre started fitting to wait\n'

# Load data and fit peaks

## Functions

In [196]:
#This binning with the error calculated in this way is not good if the input data has too few point. 
# For example, if nbins=size(x), error will be 0.
def lin_binning(x,y,nbins): #input: data x,y ; No of points you want nbins  
    #nbins+=1
    x = x[~np.isnan(x)] # remove nan from the data
    y = y[~np.isnan(y)]
    
    n, _ = np.histogram(x, bins=nbins) # n=len(x)/nbins
    meanx= np.histogram(x, bins=nbins, weights=x)[0] / n
    meany= np.histogram(x, bins=nbins, weights=y) [0]/ n
    meany2= np.histogram(x, bins=nbins, weights=y*y)[0] / n
    std_y = np.sqrt(meany2 - meany*meany)/np.sqrt(n)
    
    return meanx,meany,std_y #putput: nbins rows, 3 col


def loaddat(fpath,fname,**kwargs):
    f = os.path.join(fpath, fname);
    print(f)
    dat = np.genfromtxt(f,**kwargs) # kwargs is a dictionary and use'**' to convert to kwargs
    return dat[~np.isnan(dat[:,1]),:] 

def fitReg(dat, *args):
    if len(args)==1:# when the low and high limits is in a list
        mask = (args[0][0]< dat[:,0]) & (dat[:,0] < args[0][1])
    else: # two augs give the the two limits
        mask = (args[0]< dat[:,0]) & (dat[:,0] < args[1])
    return dat[mask,:]

# Ｃａｌｃｕｌａｔｅｄ　ｔｈｅ　ｃｏｒｒｅｓｐｉｎｄｉｎｇ　Ｑ　ｆｏｒ　ｔｈｅ　２ｔｈｅａｔａ　ｃｕｔ
mn = 1.675e-27 # neutron mass in Kg
hbar = 1.0546e-34 # in J s
coeffi0 = hbar**2/(2*mn) # in J m2
coeffi = hbar**2 / (2*mn) * 6.24181 * 10**21 * 10**20 # in meV A2

def clcq(e, ei, th2):
    f = ei - e; i = ei
    q = ((f + i - 2 * (f*i)**0.5 * np.cos(np.pi/180*th2))/coeffi)**0.5
    return q

def Qxyhw(hw, tth, Ei):
    kf = np.sqrt((Ei - hw)/2.072)
    ki = 2*np.pi/np.sqrt(81.805/Ei)
    Q = np.array([kf*np.cos(tth)-ki, kf*np.sin(tth)])
    return Q, LA.norm(Q)

def ndformfact(q):# use numpy not sp
    s,j,l,ss = q/(4*np.pi), 9/2., 6., 3/2. ### Take care of the fraction 9/2=4
    fc = (j*(j+1)-ss*(ss+1)+l*(l+1))/(3*j*(j+1)+ss*(ss+1)-l*(l+1))
    
    j0 = [0.0540,25.0293,0.3101,12.1020,0.6575,4.7223,-0.0216]
    j2 = [0.6751,18.3421,1.6272,7.2600,0.9644,2.6016,0.0150]
    
    fm = (j0[0]*np.exp(-j0[1]*s**2) + j0[2]*np.exp(-j0[3]*s**2)+j0[4]*np.exp(-j0[5]*s**2) + j0[6] 
          + s**2*(j2[0]*np.exp(-j2[1]*s**2)+j2[2]*np.exp(-j2[3]*s**2)+j2[4]*np.exp(-j2[5]*s**2)+j2[6])*fc)
    return fm**2

## Load and fit

In [197]:
# Load data
fpath = r'D:\4_Physical_Properties\Nd2Sn2O7'

dat = loaddat(fpath,'ins_cef_Nd2Sn2O7.txt', delimiter=',')
dat1 = loaddat(fpath,'ins_cef_Nd2Sn2O7_highE.txt', delimiter=',')

D:\4_Physical_Properties\Nd2Sn2O7\ins_cef_Nd2Sn2O7.txt
D:\4_Physical_Properties\Nd2Sn2O7\ins_cef_Nd2Sn2O7_highE.txt


In [198]:
def pv_func(x, A, ratio, mu, sigma,gamma):
    """
    This is the old one for Sm pyroclores.
    simga is wrongly calculated from gamma!
    """
    return A*((1-ratio)/(sigma*np.sqrt(2*np.pi))*np.exp(-(x-mu)**2/(2*sigma)**2) + \
              ratio/np.pi*gamma/((x-mu)**2+gamma**2))

def poly2(x, a, b, c):
    return a*x**2+b*x+c

log2 = np.log(2)
def pv_func(x, A, ratio, mu, gamma):
    """
    PV function
    A: intensity
    ratio: ratio between Gauss and Lorentzian
    mu: center
    gamma: FWHM for Loritian function
    """
    sigma = gamma / (2*np.sqrt(2*log2))
    return A*((1-ratio)* 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2/(2*sigma)**2) + \
              ratio* (1/np.pi * gamma/2 / ((x-mu)**2+(gamma/2)**2)))

def three_pv(x,a,b,c, A1,ratio1,mu1,gamma1, A2,ratio2,mu2,gamma2, A3,ratio3,mu3,gamma3):
    return poly2(x,a,b,c) + pv_func(x, A1, ratio1, mu1, gamma1) \
                          + pv_func(x, A2, ratio2, mu2, gamma2) \
                          + pv_func(x, A3, ratio3, mu3, gamma3)


pvmod=Model(three_pv)

params = pvmod.make_params()
#print(pvmod.param_names)
#print(pvmod.independent_vars)

### Constraining same raito and gamma for the neigboring two peaks
# params['a'].set(value=0, vary=True, min=-20, max=20)
# params['b'].set(value=0, vary=True, min=-10, max=10)
# params['c'].set(value=10, vary=True, min=-20, max=20)

# params['A1'].set(value=200., vary=True, min=0, max=None)
# params['ratio1'].set(value=0.5, vary=True, min=0, max=1)
# params['mu1'].set(value=26, vary=True, min=25, max=27)
# params['gamma1'].set(value=1.5, vary=True, min=0.1, max=2)

# params['A2'].set(value=100, vary=True, min=0, max=None)
# params['ratio2'].set(value=0.5, vary=True, min=0, max=1)
# params['mu2'].set(value=38.5, vary=True, min=37, max=39)
# params['gamma2'].set(value=1.5, vary=True, min=0.1, max=2)

# params['A3'].set(value=200, vary=True, min=0, max=None)
# params['ratio3'].set(value=0.5, vary=True, min=0, max=1, expr='ratio2')
# params['mu3'].set(value=39.8, vary=True, min=39, max=41)
# params['gamma3'].set(value=1.5, vary=True, min=0.1, max=2, expr='gamma2')

### Constraining same raito and gamma for the three peaks
params['a'].set(value=0, vary=True, min=-20, max=20)
params['b'].set(value=0, vary=True, min=-10, max=10)
params['c'].set(value=10, vary=True, min=-200, max=200)

params['A1'].set(value=200., vary=True, min=0, max=None)
params['ratio1'].set(value=0.5, vary=True, min=0, max=1)
params['mu1'].set(value=26, vary=True, min=25, max=27)
params['gamma1'].set(value=1.5, vary=True, min=0.1, max=2)

params['A2'].set(value=80, vary=True, min=0, max=None)
params['ratio2'].set(value=0.5, vary=True, min=0, max=1, expr='ratio1')
params['mu2'].set(value=38.5, vary=True, min=37, max=39)
params['gamma2'].set(value=1.5, vary=True, min=0.1, max=2, expr='gamma1')

params['A3'].set(value=200, vary=True, min=0, max=None)
params['ratio3'].set(value=0.5, vary=True, min=0, max=1, expr='ratio1')
params['mu3'].set(value=39.8, vary=True, min=39, max=41)
params['gamma3'].set(value=1.5, vary=True, min=0.1, max=2, expr='gamma1')
#print(params.valuesdict())

mask = (20<dat[:,0]) & (dat[:,0]<43)
dat = dat[mask,:]

result = pvmod.fit(dat[:,1], x=dat[:,0], params = params)
print(result.fit_report(modelpars=None, show_correl=False))
print('fitted:','\n', result.params.valuesdict())

fig, ax = plt.subplots()
ax.plot(dat[:,0], dat[:,1], marker='o')
ax.plot(dat[:,0], result.best_fit, 'r-', lw=0.5, alpha=1, label='Fit') 

x = np.linspace(dat[:,0].min(), dat[:,0].max(), num=200)
bkg = poly2(x, *[result.best_values[_] for _ in ['a', 'b', 'c']])
ax.plot(x, bkg)
ax.plot(x, bkg + pv_func(x, *[result.best_values[_+'1'] for _ in ['A', 'ratio', 'mu', 'gamma']]))
ax.plot(x, bkg + pv_func(x, *[result.best_values[_+'2'] for _ in ['A', 'ratio', 'mu', 'gamma']]))
ax.plot(x, bkg + pv_func(x, *[result.best_values[_+'3'] for _ in ['A', 'ratio', 'mu', 'gamma']]))

#ax.set_ylim([-2,8])
plt.show()


[[Model]]
    Model(three_pv)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 97
    # data points      = 101
    # variables        = 11
    chi-square         = 10928.7900
    reduced chi-square = 121.431000
    Akaike info crit   = 495.087571
    Bayesian info crit = 523.853896
[[Variables]]
    a:       0.07132733 +/- 0.03253267 (45.61%) (init = 0)
    b:      -4.14708042 +/- 2.04691594 (49.36%) (init = 0)
    c:       70.2770448 +/- 31.2584461 (44.48%) (init = 10)
    A1:      233.131387 +/- 30.7447064 (13.19%) (init = 200)
    ratio1:  0.48133363 +/- 0.18419119 (38.27%) (init = 0.5)
    mu1:     25.8753833 +/- 0.01501440 (0.06%) (init = 26)
    gamma1:  0.98125554 +/- 0.04791279 (4.88%) (init = 1.5)
    A2:      63.8681712 +/- 10.6090472 (16.61%) (init = 80)
    ratio2:  0.48133363 +/- 0.18419119 (38.27%) == 'ratio1'
    mu2:     38.1691785 +/- 0.09502433 (0.25%) (init = 38.5)
    gamma2:  0.98125554 +/- 0.04791279 (4.88%) == 'gamma1'
    A3:      118

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [199]:
(27-24.5)*(197-12)/2, (39.1-36.3)*(46.4-9)/2, (41.77-37.86)*(101-9)/2 

(231.25, 52.36000000000008, 179.86000000000018)

In [200]:
a1, e1 = result.best_values['A1'], result.best_values['mu1']
a2, e2 = result.best_values['A2'], result.best_values['mu2']
a3, e3 = result.best_values['A3'], result.best_values['mu3']

a1 = a1/integrate.quad(ndformfact, clcq(e1, 60, 3), 4)[0]
a2 = a2/integrate.quad(ndformfact, clcq(e2, 60, 3), 4)[0]
a3 = a3/integrate.quad(ndformfact, clcq(e3, 60, 3), 4)[0]

a1, a2, a3, a1/a3, a2/a3

(111.68111988983766,
 46.91037745058989,
 94.50374474538336,
 1.181763962800992,
 0.49638644031491175)

In [208]:
print(*result.best_values)

a b c A1 ratio1 mu1 gamma1 A2 ratio2 mu2 gamma2 A3 ratio3 mu3 gamma3


In [215]:
es = np.linspace(20,45,num=75)
Is = three_pv(es, 0, 0, 0, 
              result.best_values['A1']/integrate.quad(ndformfact, clcq(e1, 60, 3), 4)[0], result.best_values['ratio1'], result.best_values['mu1'], result.best_values['gamma1'], 
              result.best_values['A2']/integrate.quad(ndformfact, clcq(e2, 60, 3), 4)[0], result.best_values['ratio2'], result.best_values['mu2'], result.best_values['gamma2'], 
              result.best_values['A3']/integrate.quad(ndformfact, clcq(e3, 60, 3), 4)[0], result.best_values['ratio3'], result.best_values['mu3'], result.best_values['gamma3'], 
        )
plt.figure()
plt.plot(es,Is)
np.vstack([es, Is]).T

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[20.        ,  0.27030935],
       [20.33783784,  0.30148282],
       [20.67567568,  0.33870675],
       [21.01351351,  0.38368064],
       [21.35135135,  0.43874631],
       [21.68918919,  0.50720182],
       [22.02702703,  0.59380897],
       [22.36486486,  0.70564362],
       [22.7027027 ,  0.85359845],
       [23.04054054,  1.05547619],
       [23.37837838,  1.34584688],
       [23.71621622,  1.82410541],
       [24.05405405,  2.87333322],
       [24.39189189,  5.819796  ],
       [24.72972973, 13.83597665],
       [25.06756757, 31.1241847 ],
       [25.40540541, 58.59300158],
       [25.74324324, 86.65566884],
       [26.08108108, 81.90294077],
       [26.41891892, 51.96662102],
       [26.75675676, 26.44194699],
       [27.09459459, 11.45686927],
       [27.43243243,  4.91748367],
       [27.77027027,  2.58817189],
       [28.10810811,  1.73554047],
       [28.44594595,  1.32221401],
       [28.78378378,  1.06372728],
       [29.12162162,  0.88408519],
       [29.45945946,

In [ ]:
"""
Notes:

Why the high energy peaks are broader!
"""


# Fit with Spectre

## Using subprocess 

In [131]:
# Ｄｅｆｉｎｅ　ｆｕｎｓ　ｎｏｔ　ｃｌａｓｓ　ｆｏｒ　ｃａｌ：　
# ｗｈｅｎ　ｕｓｉｎｇ，　ｆｉｒｓｔ　ｒｕｎ　ｃｏｎｓｏｌｅ　ｔｈｅｎ　ｒｕｎ　ｏｔｈｅｒｓ　ｗｉｔｈ　ｔａｋｉｎｇ　ｔｈｅ　ｐｒｏｃｅｓｓ　ａｓ　ｉｎｐｕｔ
# Note: 'stdin.write' seem not waiting for the finish of the run of evrey cmd

''' Set which ion manually in Spectre first!!!!
console(set_pars): call Spectre via cmd, the set parameters set_pars: Bqk, B field; 
cal_nd(set_pars,ndfpath): cal crystal field neutron scatering, and save to file;
fit_nd(set_pars,cmds,ndfpath):input data, fit and save display to file by Python(Spectre also yields a file)
cal_mh(set_pars,cmds):calcualted mh at a temperature, field direction and seve to file by Spectre
''' 
import subprocess
import os
def console(**kwargs):
    set_pars = []
    if kwargs:
        for kw in kwargs:
            if kw=='set_pars': set_pars=kwargs[kw]
                
    #start Spectre
    proc = subprocess.Popen(['cmd.exe','-u','stdbuf'], bufsize=0, stdin = subprocess.PIPE, stdout = subprocess.PIPE, encoding='utf8')
    proc.stdin.write('D: \n')
    proc.stdin.write('cd D:/2_Neutron_scattering/0_Spectre2018 \n')
    proc.stdin.write('spectre.exe \n')
    #instea of above for calling Spectre, blow lines also works
#   proc.stdin.write('D: \n'+'cd D:\9_Program files\crystal_field \n'+'spectre.exe \n')
   
    if set_pars:
        for par in set_pars:
            proc.stdin.write(par+'\n')
        proc.stdin.write('\n')          

    return(proc)

def start_setPar_fit(fitcmds, **kwargs):
    set_pars = []
    save_path = []
    if kwargs:
        for kw in kwargs:
            if kw=='set_pars': set_pars=kwargs[kw]
            if kw=='save_path': save_path=kwargs[kw]    
    #start Spectr，set CEF parameters and do fitting
    proc = subprocess.Popen('cmd.exe', stdin = subprocess.PIPE, stdout = subprocess.PIPE, encoding='utf8')
    if set_pars:
        output = proc.communicate('D: \n'+'cd D:/2_Neutron_scattering/0_Spectre2018 \n'+'spectre.exe \n'+ \
                                  '\n'.join(set_pars)+'\n'+'\n'+ \
                                  '\n'+'4'+'\n'+'\n' +'\n'.join(fitcmds) +'\n'+'\n'+'\n'+'y'+'\n')[0]
    else:
        output = proc.communicate('D: \n'+'cd D:/2_Neutron_scattering/0_Spectre2018 \n'+'spectre.exe \n'+ \
                                  '\n'+'4'+'\n'+'\n'+'\n'.join(cmds)+'\n'+'\n'+'\n'+'y'+'\n')[0]
    if save_path:    
        with open(save_path,'w+') as f:
            f.writelines(output)
            f.close()   
    return


def fit_cef(proc,cmds,fpath,fname):
    output = proc.communicate('\n'+'4'+'\n'+'\n'+'\n'.join(cmds)+'\n'+'\n'+'\n'+'y'+'\n')[0]
    #meaning of the cmds: \n enter main menu, 4\n select fit, \n not going to fit existing data, 
    #\n.join(cmds) input data, \n,\n input v return to E,C,R,V.. menu, \n fit,y \n save
    with open(os.path.join(fpath,fname),'w+') as f: # append or read, if not exist then creat
        f.writelines(output)
        f.close()     
    return proc, output

def cal_mh(proc,cmds):# cmds: ['2', '0 0 5', 'fname.txt'] (temp, field,filename)
    mh_output = proc.communicate('\n'+'7'+'\n'+cmds[0]+'\n'+cmds[1]+'\n'+'y'+'\n'+cmds[2]+'\n')[0]
    #meaning:\n enter main menu,7\n selcte cal MH, 2\n 2K, 005\n field direction,y\n save, file,txt filename
    #print mh_output
    return

def cal_mh1(proc,temp,field,fpath):
    proc.stdin.write('\n')
    proc.stdin.write('7 \n')
    proc.stdin.write(str(temp))
    proc.stdin.write('\n')
    proc.stdin.write(field)
    proc.stdin.write('\n')
    proc.stdin.write('y \n')
    proc.stdin.write(fpath)
    proc.stdin.write('\n')
    proc.communicate()
    proc.terminate()
    #meaning: \n enter main menu,7\n selcte cal MH, 2\n 2K, 005\n field direction,y\n save, file,txt filename
    #print mh_output
    return

def cal_cp(proc,T_range_npts,fname):# The last two input must be strings
    proc.communicate('\n'+ '5 \n' + T_range_npts+'\n' +'y \n' +fname+'\n')
    return

def cal_xt(proc,maxT,fname):# The last two input must be strings
    proc.communicate('\n'+ '6 \n' + maxT+'\n' +'y \n' +fname+'\n')
    return


In [138]:
start_pars =  ['1,49.2',  '2,408.9', '3,148.1', '8,121.6', '12,-98', '15,139']# Nd2Zr2O7 pars
#start_pars  = ['1,53.9',  '2,378',   '3,118',   '8,141',   '12,-88', '15,157'] # Scaled from Pr2Sn2O7 by Pricip.
ndzro = np.array([49.2,  408.9, 148.1, 121.6, -98, 139])

# scale Bs of Nd2Zr2O7 by total energy splitting
idx = [1,2,3,8,12,15]
Bs_scaled = ndzro*113/106 
start_pars = [str(i)+','+str(b) for i,b in zip(idx, Bs_scaled)]

# scale Bs of Nd2Zr2O7 by piont charge calculation
point_charge_Sn_div_Zr = np.array([1.30023928, 1.03865862, 1.02587187, 0.94203426, 0.95328389,0.99249977]) # Be careful: order of Bs are different in Specte and PyCrystalField
Bs_scaled = np.round(point_charge_Sn_div_Zr * ndzro, decimals=0)
start_pars = [str(i)+','+str(b) for i,b in zip(idx, Bs_scaled)]

print(Bs_scaled)
print(start_pars)

proc = console(set_pars = start_pars)
print(proc.communicate()[0])

[ 64. 425. 152. 115. -93. 138.]
['1,64.0', '2,425.0', '3,152.0', '8,115.0', '12,-93.0', '15,138.0']
Microsoft Windows [Version 10.0.17763.2183]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\jxu\0_MyIpython\Crystal_field>D: 

D:\>cd D:/2_Neutron_scattering/0_Spectre2018 

D:\2_Neutron_scattering\0_Spectre2018>spectre.exe 





                  CEF Parameters (meV):

    1  B(2,0) =  0.640E+02  2  B(4,0) =  0.425E+03  3  B(6,0) =  0.152E+03
    4  B(2,1)c =  0.000E+00  16  B(2,1)s =  0.000E+00
    5  B(2,2)c =  0.000E+00  17  B(2,2)s =  0.000E+00
    6  B(4,1)c =  0.000E+00  18  B(4,1)s =  0.000E+00
    7  B(4,2)c =  0.000E+00  19  B(4,2)s =  0.000E+00
    8  B(4,3)c =  0.115E+03  20  B(4,3)s =  0.000E+00
    9  B(4,4)c =  0.000E+00  21  B(4,4)s =  0.000E+00
   10  B(6,1)c =  0.000E+00  22  B(6,1)s =  0.000E+00
   11  B(6,2)c =  0.000E+00  23  B(6,2)s =  0.000E+00
   12  B(6,3)c = -0.930E+02  24  B(6,3)s =  0.000E+00
   13  B(6,4)c =  0.000E+00  25  B(6,4)s =  0.000E+00

In [44]:
proc.communicate()[0]

In [109]:
# fitting
fitcmd = ['e','1 2 0 0.1', '1 3 25.9 0.1','1 4 25.9 0.2', '1 5 38.0 0.2','1 6 38.0 0.2',
          '1 7 39.8 0.2','1 8 39.8 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\n',
          'c','1 3','1 4\n', 'c','1 5','1 6\n', 'c','1 7','1 8\n',
          'r', '5 1 3','5 1 7','1.5 0.1',
               '5 1 5','5 1 7','0.5 0.05\n',
          'v','1','2','3','8','12','15']

# Exchanged peaks poisitons 1-5, 1-6 vs 1-7, 1-8
fitcmd1 = ['e','1 2 0 0.1', '1 3 25.9 0.2','1 4 25.9 0.2', '1 5 39.8 0.2','1 6 39.8 0.2',
          '1 7 38.0 0.2','1 8 38.0 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\n',
          'c','1 3','1 4\n', 'c','1 5','1 6\n', 'c','1 7','1 8\n',
          'r', '5 1 3','5 1 5','1.2 0.1',
               '5 1 7','5 1 5','0.5  0.05\n',
          'v', '1','2','3','8','12','15']

In [119]:
fpath = r'D:/2_Neutron_scattering/0_Spectre2018\Nd2Sn2O7'
fname = 'nso_CEF_fit.txt' # file to store all the fitting output
proc = console(set_pars = start_pars)
proc, output = fit_cef(proc,fitcmd1,fpath,fname) # exery time the fit result is different!
#proc.poll() # check alinve subprocess: none or 0 (no alive)

In [120]:
print(output)

Microsoft Windows [Version 10.0.17763.2183]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\jxu\0_MyIpython\Crystal_field>D: 

D:\>cd D:/2_Neutron_scattering/0_Spectre2018 

D:\2_Neutron_scattering\0_Spectre2018>spectre.exe 





                  CEF Parameters (meV):

    1  B(2,0) =  0.500E+02  2  B(4,0) =  0.448E+03  3  B(6,0) =  0.136E+03
    4  B(2,1)c =  0.000E+00  16  B(2,1)s =  0.000E+00
    5  B(2,2)c =  0.000E+00  17  B(2,2)s =  0.000E+00
    6  B(4,1)c =  0.000E+00  18  B(4,1)s =  0.000E+00
    7  B(4,2)c =  0.000E+00  19  B(4,2)s =  0.000E+00
    8  B(4,3)c =  0.931E+02  20  B(4,3)s =  0.000E+00
    9  B(4,4)c =  0.000E+00  21  B(4,4)s =  0.000E+00
   10  B(6,1)c =  0.000E+00  22  B(6,1)s =  0.000E+00
   11  B(6,2)c =  0.000E+00  23  B(6,2)s =  0.000E+00
   12  B(6,3)c = -0.104E+03  24  B(6,3)s =  0.000E+00
   13  B(6,4)c =  0.000E+00  25  B(6,4)s =  0.000E+00
   14  B(6,5)c =  0.000E+00  26  B(6,5)s =  0.000E+00
   15  B(6,6)c =  0.143E+03  27  B(6,6)s =  0

# Use wexpect (writing a logfile is not working)

## functions

In [50]:
# Ｓｔａｒｔ　Ｓｐｅｃｔｒｅ　ａｎｄ　ｓｅｔ　ＣＥＦ　ｐａｒａｍｅｔｅｒｓ
import wexpect
def console(**kwargs):
    # start Spectre
    proc = wexpect.spawn('cmd.exe')
    
    set_pars = []
    if kwargs:
        for kw in kwargs:
            if kw=='set_pars': 
                set_pars=kwargs[kw]   
            if kw=='logfile':
                proc.logfile = kwargs[kw] 
                print(proc.logfile)
                
    proc.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])
    # set parameters
    if set_pars:
        proc.writelines('\r\n'.join(set_pars)+'\r\n')
        proc.write('\r\n')          
    return(proc)

# Ｆｉｔ　ＣＥＦ　ｄａｔａ
def fit_cef(proc,cmds):
    proc.writelines(['\r\n'+'4'+'\r\n'+'\r\n'])
    for cmd in cmds:
        proc.sendline(cmd)
    proc.writelines(['\r\n','\r\n','\r\n','y','\r\n'])

#def fit_cef(proc,cmds): # very slow or not working
#    proc.writelines('\r\n'+'4'+'\r\n'+'\r\n'+'\r\n'.join(cmds)+'\r\n'+'\r\n'+'\r\n'+'y'+'\r\n')
    
#　Calculate　Chi-T
def cal_xt(proc,maxT,fname):# The last two input must be strings
    proc.writelines('\r\n'+ '6 \r\n' + maxT+'\r\n' +'y \r\n' +fname+'\r\n')
    return    
    
#　Calculate Cp   
# The five Cp functions all work  
def cal_cp(child,T_range_npts,fname):# The last two input must be strings
    child.writelines('\r\n'+'5'+'\r\n'+ T_range_npts+'\r\n'+ 'y'+'\r\n' +fname+'\r\n')
    return

In [152]:
nd_pars = ['1,49.2',  '2,408.9', '3,148.1', '8,121.6', '12,-98', '15,140']# Nd2Zr2O7 pars

fitcmd = ['e','1 2 0 0.1', '1 3 25.9 0.1', '1 4 25.9 0.2', '1 5 38.0 0.2','1 6 38.0 0.2',
          '1 7 39.8 0.2','1 8 39.8 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\r\n',
          'c','1 3','1 4\r\n', 'c','1 5','1 6\r\n', 'c','1 7','1 8\r\n',
          'r', '5 1 3','5 1 7','1.5 0.1',
               '5 1 5','5 1 7','0.5 0.05\r\n',
          'v','1','2','3','8','12','15']

# Exchanged peaks poisitons 1-5, 1-6 vs 1-7, 1-8
fitcmd1 = ['e','1 2 0 0.1', '1 3 25.9 0.2','1 4 25.9 0.2', '1 5 39.8 0.2','1 6 39.8 0.2',
          '1 7 38.0 0.2','1 8 38.0 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\r\n',
          'c','1 3','1 4\r\n', 'c','1 5','1 6\r\n', 'c','1 7','1 8\r\n',
          'r', '5 1 3','5 1 5','1.2 0.1',
               '5 1 7','5 1 5','0.5  0.05\r\n',
          'v', '1','2','3','8','12','15']

#with open('D:/2_Neutron_scattering/0_Spectre2018/Nd2Sn2O7/logfile.txt','wb') as fout:
#    proc = console(set_pars=nd_pars, logfile=fout)

#    fit_cef(proc, fitcmd)
    #proc.writelines(['\r\n', '\r\n', '99\r\n']) # close spectre.exe after fitting （works only after the fitting finishes）

    #proc.expect(wexpect.EOF)
    

In [145]:
#fit_cef(proc,fitcmd)
proc.writelines(['\r\n', '\r\n', '99\r\n']) # close spectre.exe aftter fitting （works only after the fitting finishes）

EOF: The pipe is being closed.

In [31]:
proc.close()

## withour defining functions

In [57]:
with open('D:/2_Neutron_scattering/0_Spectre2018/Nd2Sn2O7/logfile.txt','wb') as fout:
    proc = wexpect.spawn('cmd.exe')              
    proc.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])
    
    # Set parameters
    proc.writelines('\r\n'.join(nd_pars)+'\r\n')
    proc.write('\r\n')          

    # Fit
    proc.writelines(['\r\n'+'4'+'\r\n'+'\r\n'])
    for cmd in fitcmd1:
        proc.sendline(cmd)
    proc.writelines(['\r\n','\r\n','\r\n'])
    
    # Expect
    ind = proc.expect(['IFAIL =  0', r'IFAIL =  [12345]', wexpect.TIMEOUT, wexpect.EOF, 'best-fit1'], timeout=100) # after "=" is two spaces
    proc.expect(['Save these best-fit parameters ? (N) :', wexpect.TIMEOUT, wexpect.EOF], timeout=1)
    if ind==0:
        print('************** Succusess ******************')
    elif ind==1:
        print('************* Fit Failed *******************')
        time.sleep(1)
        print(proc.before, end='')
        print(proc.after,  end='')
    elif ind==2:
        print('************** Time out ********************')
    elif ind==3:
        print('************** No match ********************')
    else:
        print(proc.before, end='')
        print(proc.after,  end='')
    
    proc.writelines(['y','\r\n']) # Save results.dat
    
    # Close
    proc.writelines(['\r\n', '\r\n', '99\r\n']) # close spectre.exe after fitting
    proc.close()

************* Fit Failed *******************



    CEF Parameter (meV)

  1  B(2,0) =  5.094E+01 +/- 3.980E+02
  2  B(4,0) =  4.349E+02 +/- 5.503E+02
  3  B(6,0) =  1.471E+02 +/- 1.009E+02
  4  B(2,1)c =  0.000E+00 +/- 0.000E+00
  5  B(2,2)c =  0.000E+00 +/- 0.000E+00
  6  B(4,1)c =  0.000E+00 +/- 0.000E+00
  7  B(4,2)c =  0.000E+00 +/- 0.000E+00
  8  B(4,3)c =  9.034E+01 +/- 7.842E-07
  9  B(4,4)c =  0.000E+00 +/- 0.000E+00
 10  B(6,1)c =  0.000E+00 +/- 0.000E+00
 11  B(6,2)c =  0.000E+00 +/- 0.000E+00
 12  B(6,3)c = -1.013E+02 +/- 1.014E+02
 13  B(6,4)c =  0.000E+00 +/- 0.000E+00
 14  B(6,5)c =  0.000E+00 +/- 0.000E+00
 15  B(6,6)c =  1.481E+02 +/- 3.349E+02
 16  B(2,1)s =  0.000E+00 +/- 0.000E+00
 17  B(2,2)s =  0.000E+00 +/- 0.000E+00
 18  B(4,1)s =  0.000E+00 +/- 0.000E+00
 19  B(4,2)s =  0.000E+00 +/- 0.000E+00
 20  B(4,3)s =  0.000E+00 +/- 0.000E+00
 21  B(4,4)s =  0.000E+00 +/- 0.000E+00
 22  B(6,1)s =  0.000E+00 +/- 0.000E+00
 23  B(6,2)s =  0.000E+00 +/- 0.000E+00
 24  B(6,3

## Test wexpect

In [6]:

child = wexpect.spawn('cmd.exe')
child.expect('>')
print(child.before)
print(child.after, end='')

child.sendline('D:')
child.expect('>')
print(child.before)
print(child.after, end='')

child.sendline('cd D:/2_Neutron_scattering/0_Spectre2018/')
child.expect('>')
print(child.before)
print(child.after, end='')

child.sendline('spectre.exe\n')
time.sleep(2)
child.kill()
# Exit from cmd
#child.sendline('exit')
# Waiting for cmd termination.
#child.wait()

Microsoft Windows [Version 10.0.17763.2183]
(c) 2018 Microsoft Corporation. All rights reserved.

C:\Users\jxu\0_MyIpython\Crystal_field
>D:

D:\
>cd D:/2_Neutron_scattering/0_Spectre2018/

D:\2_Neutron_scattering\0_Spectre2018
>

In [51]:
child = wexpect.spawn('cmd.exe')
child.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])
time.sleep(15)
child.writelines(['\r\n', '99\r\n']) # Close spectre.exe
child.kill()

In [52]:
child.writelines(['\r\n', '99\r\n']) # Close spectre.exe

EOF: The pipe is being closed.

# Random starting parameters

## Fit

In [2]:
fitcmd = ['e','1 2 0 0.1', '1 3 25.9 0.1', '1 4 25.9 0.2', '1 5 38.0 0.2','1 6 38.0 0.2',
          '1 7 39.8 0.2','1 8 39.8 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\r\n',
          'c','1 3','1 4\r\n', 'c','1 5','1 6\r\n', 'c','1 7','1 8\r\n',
          'r', '5 1 3','5 1 7','1.2 0.1',
               '5 1 5','5 1 7','0.5 0.05\r\n',
          'v','1','2','3','8','12','15']

# Exchanged peaks poisitons 1-5, 1-6 vs 1-7, 1-8
fitcmd1 = ['e','1 2 0 0.1', '1 3 25.9 0.2','1 4 25.9 0.2', '1 5 39.8 0.2','1 6 39.8 0.2',
          '1 7 38.0 0.2','1 8 38.0 0.2', '1 9 110.0 0.5','1 10 110.0 0.5\r\n',
          'c','1 3','1 4\r\n', 'c','1 5','1 6\r\n', 'c','1 7','1 8\r\n',
          'r', '5 1 3','5 1 5','1.2 0.1',
               '5 1 7','5 1 5','0.5  0.05\r\n',
          'v', '1','2','3','8','12','15']

In [12]:
# Ｌｏｏｐｓ　ｗｉｔｈ　Ｒａｎｄｏｍ ｐａｒａｍｅｔｅｒ　ｗｉｔｈｉｎ　ａ　ｃｅｒａｔａｉｎ　ｒａｎｇｅ
# copy result.dat files
cmds = fitcmd
fpath = r'D:\2_Neutron_scattering\0_Spectre2018\Nd2Sn2O7\Output1'
#cmds = fitcmd1
#fpath = r'D:\2_Neutron_scattering\0_Spectre2018\Nd2Sn2O7\Output0'  # exchanged the two close levels

# Open file store starting Bs
f1 = open(os.path.join(fpath,'Bs_starting.txt'), "a+") # file to store the starting B parameters

# Start Spectre
idxs = ['1','2','3','8','12','15']
Bs_scaled = np.array([64., 425., 152., 115., -93., 138.])

start = time.time()
proc = wexpect.spawn('cmd.exe')              
proc.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])

iters=100
max_iters = 1000
while iters<max_iters:
    # Set initio parameters (randoms)
    cefp = list(map(str, Bs_scaled * (np.random.rand(6)+0.5)))

    for idx, par in zip(idxs, cefp):
        proc.write(idx+' '+par+'\r\n')
    proc.write('\r\n') 
    
    #print('\n', iters, cefp)
    
    # Write initial par to file
    f1.write(' '.join(cefp))
    f1.write('\n')
    
    # Fit
    proc.writelines(['\r\n'+'4'+'\r\n'+'\r\n'])
    for cmd in cmds:
        proc.sendline(cmd)
    proc.writelines(['\r\n','\r\n','\r\n'])
    
    ind = proc.expect(['IFAIL =  0', r'IFAIL =  [12345]', wexpect.TIMEOUT, wexpect.EOF], timeout=200) # after "=" is two spaces
    if ind==0:
        print('Succusess')
    #elif ind==1: 
        #print(proc.before, end='')
        #print(proc.after,  end='')
    
    shutil.copy(r'D:/2_Neutron_scattering/0_Spectre2018/RESULT.DAT', os.path.join(fpath,'RESULT%d.DAT'%iters))
    
    iters = iters+1 

f1.close()

end = time.time()
print(end - start)


32255.174607753754


In [21]:
proc.writelines(['\r\n', '\r\n', '99\r\n']) # close spectre.exe after fitting
proc.close()

EOF: The pipe is being closed.

## Load the Bs and chi2

In [14]:
fname='RESUL*.DAT'

fnames=[] # store all the right file names in the folder
for file in os.listdir(fpath):
    if fnmatch.fnmatch(file, fname):      
        fnames.append(file)
#print fnames[:2]

# write them to a file
final = open(os.path.join(fpath,'Bs_chi.txt'),"w") # append writing
final.write(' '.join(['B20','B40','B60','B43','B63','B66','Chi','\n'])) # write headers

for fname in fnames:      
    fname = os.path.join(fpath,fname)
    f = open(fname, 'r')
    #print(fname)
    ftext = f.read()
    f.close()
    m1 = re.findall("(B\([246],[036]\)[\ *c]?)\s+=\s+(-?\d+\.\d+E[-+]\d+)", ftext)
    m2 = re.findall("(CHI SQUARED)\s+=\s+(\d+\.\d+)", ftext)
    if m1 and m2:
        #print m1, m2 
        final.write(' '.join([m1[0][1],m1[1][1],m1[2][1],m1[3][1],m1[4][1],m1[5][1],m2[0][1],'\n']))
    else:
        final.write(' '.join(['Nan','Nan','Nan','Nan','Nan','Nan','Nan','\n']))# Write 'Nan' for bad fittings with not futll output
        
final.close()

In [221]:
# load fitted　CEF parameters　and chi2
fpath = r'D:\2_Neutron_scattering\0_Spectre2018\Nd2Sn2O7\Output1'
dat = np.genfromtxt(os.path.join(fpath,'Bs_chi.txt'), skip_header=1)
chi = dat[:,6] # used for plotting Bs_starting.txt
print(dat.shape)

idx_min = np.nanargmin(dat[:,-1])
print(idx_min)
dat[idx_min,:]


(1000, 7)
110


array([ 3.450e+01,  4.454e+02,  1.387e+02,  7.446e+01, -1.082e+02,
        1.498e+02,  4.193e-01])

In [222]:
# Set the best-fit parameters to Spectre
proc = wexpect.spawn('cmd.exe')              
proc.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])

# Set initio parameters (randoms)
for idx, par in zip(idxs, dat[idx_min, :-1]):
    proc.write(idx+' '+str(par)+'\r\n')
proc.write('\r\n') 

In [223]:
proc.writelines(['\r\n', '99\r\n']) # close spectre.exe after setting
proc.close()

## Plot

In [16]:
# Plot the CEF parameters vs chi
# Later can try to link those figure(mouse coursor), we can refer the expamle
# http://stackoverflow.com/questions/13306519/get-data-from-plot-with-matplotlib

label = ['B20','B40','B60','B43','B63','B66']

import matplotlib.cm as cm
colors = cm.rainbow(np.linspace(0, 1, len(dat)))# plot pars of the same set with the same color
    
fig, axs = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=True, figsize=(12,8))
for idx, ax in enumerate(axs.reshape(-1)):
    for x, y, c in zip(dat[:,idx], dat[:,6], colors):
        ax.scatter(x, y, color=c)
    ax.set_xlabel(label[idx],fontsize=14)
    ax.autoscale_view(True,True,True)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True)
    if idx==0 or idx==3:
        ax.set_ylabel(r'$\chi^2$', fontsize = 14)        
    #ax.plot(dat[:,idx],dat[:,6],'o')

#axs[0,0].set_ylim([-0.5,5])    
#axs[0,1].set_xlim([350,450])
#axs[0,2].set_xlim([50,250])   
#plt.suptitle('Fitted CEF parameters (meV) vs Chi')   
#fig.savefig(os.path.join(fpath,'szo_cef_random.pdf'), bbox_inches="tight",verbose=True)  
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Susceptibility and Cp

In [60]:
# Set the best-fit parameters to Spectre
proc = wexpect.spawn('cmd.exe')              
proc.writelines(['D:\r\n', 'cd D:/2_Neutron_scattering/0_Spectre2018/\r\n', 'spectre.exe\r\n'])

# Set initio parameters (randoms)
# for idx, par in zip(idxs, dat[idx_min, :-1]):
#     proc.write(idx+' '+str(par)+'\r\n')
# proc.write('\r\n') 
#proc.writelines(['\r\n','\r\n', '99\r\n']) # close spectre.exe after fitting
#proc.close()

In [25]:
proc.close()

In [61]:
try:
    os.remove(r'D:/2_Neutron_scattering/0_Spectre2018/nso_cp.txt')
    os.remove(r'D:/2_Neutron_scattering/0_Spectre2018/nso_xt.txt')
except:
    pass

cal_cp(proc,'2 300 100','nso_cp.txt')
cal_xt(proc,'2 350 100','nso_xt.txt')

In [62]:
proc.writelines(['\r\n','\r\n', '99\r\n']) # close spectre.exe after fitting
proc.close()

ExceptionPexpect: Child has not been terminated even after it was killed.

In [190]:
# read and plot the Cp and XT data
def load_xt(fname):    
    temp = np.genfromtxt(fname, usecols=(0,1,2,3), 
                         converters = {1: lambda s: float(s.replace(b'D',b'E')),
                                       2: lambda s: float(s.replace(b'D',b'E')),
                                       3: lambda s: float(s.replace(b'D',b'E'))} )
    return temp

# load data
fnames = [r'D:/2_Neutron_scattering/0_Spectre2018/nso_cp.txt',
          r'D:/2_Neutron_scattering/0_Spectre2018/nso_xt.txt',
          r'D:\4_Physical_Properties\Nd2Sn2O7\Nd2Sn2O7_chiT.txt',
          r'D:\4_Physical_Properties\Nd2Sn2O7\Nd2Sn2O7_powder_1stBatch_mt_1T.txt',
          r'D:\4_Physical_Properties\Nd2Sn2O7\Nd2Sn2O7_chiT_1T_inOrigin.txt',
          r'D:\4_Physical_Properties\Nd2Sn2O7...']

cp = np.genfromtxt(fnames[0])
xt = load_xt(fnames[1]) # calulated

xt1 = np.genfromtxt(fnames[2]) # data
#print(xt1[:5])
#xt1[:,2] = xt1[:,2]/10000 /0.02432 *637.8998/2


In [192]:
# fast check plot
fig, axs = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(12,4))

# plot Cp
axs[0].plot(cp[:,0],cp[:,1])
#axs[0].plot(szocp1[:,0],szocp1[:,1],'ro')
axs[0].set_xlabel(r'T(K)')
axs[0].set_ylabel(r'Cp (J/K/mol Sm)')
# plot x-T
axs[1].plot(xt[:,0],xt[:,1],label=r'$\chi_{\rm z}$')
axs[1].plot(xt[:,0],xt[:,2],label=r'$\chi_{\rm xy}$')
axs[1].plot(xt[:,0],xt[:,3],label=r'$\chi_{\rm powder}$')
axs[1].set_xlabel(r'T(K)')
axs[1].set_ylabel(r'$\chi$(emu/mol Nd)')

# plot 1/x-T
axs11 = axs[1].twinx()
axs11.grid(b=False)
axs11.plot(xt1[:,0],1./xt1[:,1],'o',color='black', label='Measured')
axs11.plot(xt[:,0], 1./xt[:,3],color='r', label='Calculated')
# Make the y-axis labeld tick labels match the line color.
for tl in axs11.get_yticklabels():
    tl.set_color('black')
axs11.set_ylabel(r'$1/\chi_{\rm powder}$',color='black')

axs[0].legend(fontsize=14)
axs[1].legend(fontsize=14,loc='best')
axs11.legend(fontsize=14,loc='best',numpoints=1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [163]:
plt.close('all')